In [1]:
import os
from PIL import Image
import numpy as np
from tqdm import tqdm
from skimage.filters import threshold_otsu, threshold_sauvola

ModuleNotFoundError: No module named 'PIL'

In [ ]:
from PIL import Image
import os

sizes = set()

for img in os.listdir("/data/images"):
    im = Image.open(os.path.join("/data/images", img))
    sizes.add(im.size)

print(sizes)


{(512, 512), (256, 256)}


In [ ]:
class BrainMRISegmentation:
    def __init__(self, root_dir, target_size=512):
        self.img_dir = os.path.join(root_dir, "images")
        self.mask_dir = os.path.join(root_dir, "masks")

        self.images = sorted([f for f in os.listdir(self.img_dir) if not f.startswith('.')])
        self.masks = sorted([f for f in os.listdir(self.mask_dir) if not f.startswith('.')])

        assert len(self.images) == len(self.masks), "Image-mask mismatch!"

        for imf, mf in zip(self.images, self.masks):
            assert os.path.splitext(imf)[0] == os.path.splitext(mf)[0], \
                f"Filename mismatch: {imf} vs {mf}"

        self.target_size = target_size

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        fn = self.images[idx]

        img_path = os.path.join(self.img_dir, fn)
        mask_path = os.path.join(self.mask_dir, fn)

        img = Image.open(img_path).convert("L")
        mask = Image.open(mask_path).convert("L")

        # Resize if needed
        img = img.resize((self.target_size, self.target_size), resample=Image.BILINEAR)
        mask = mask.resize((self.target_size, self.target_size), resample=Image.NEAREST)

        img_arr = np.asarray(img).astype(np.uint8)
        mask_arr = np.asarray(mask).astype(np.uint8)

        mask_bin = (mask_arr > 127).astype(np.uint8)

        return img_arr, mask_bin, fn


## Otsu Segmentation

In [ ]:
def apply_otsu(image):
    threshold = threshold_otsu(image)
    binary = (image > threshold).astype(np.uint8)
    return binary


## Sauvola

In [ ]:
def apply_sauvola(image, window_size=51, k=0.2):
    thresh = threshold_sauvola(image, window_size=window_size, k=k)
    binary = (image > thresh).astype(np.uint8)
    return binary


In [ ]:
def tune_sauvola(dataset, window_sizes, k_values):

    # Cache dataset in memory (important for speed)
    cached_data = [dataset[i] for i in range(len(dataset))]

    best_score = 0
    best_params = None

    for ws in window_sizes:
        for k in k_values:

            dice_scores = []

            for img, mask, _ in tqdm(cached_data,
                                     desc=f"WS={ws}, k={k}",
                                     leave=False):

                pred = apply_sauvola(img, window_size=ws, k=k)
                d = dice_score(pred, mask)
                dice_scores.append(d)

            mean_dice = np.mean(dice_scores)

            print(f"Window={ws}, k={k}, Mean Dice={mean_dice:.4f}")

            if mean_dice > best_score:
                best_score = mean_dice
                best_params = (ws, k)

    print("\nBest Sauvola Params:", best_params, "Dice:", best_score)
    return best_params, cached_data


## Metrics

In [ ]:
def dice_score(pred, target, eps=1e-8):
    intersection = np.logical_and(pred, target).sum()
    union = pred.sum() + target.sum()
    if union == 0:
        return 1.0
    return (2.0 * intersection + eps) / (union + eps)


def jaccard_index(pred, target, eps=1e-8):
    intersection = np.logical_and(pred, target).sum()
    union = np.logical_or(pred, target).sum()
    if union == 0:
        return 1.0
    return (intersection + eps) / (union + eps)


## Evaluation

In [ ]:
def evaluate(cached_data, sauvola_params):

    ws, k = sauvola_params

    otsu_dice, otsu_jacc = [], []
    sau_dice, sau_jacc = [], []

    for img, mask, _ in tqdm(cached_data):

        # Otsu
        pred_otsu = apply_otsu(img)
        otsu_dice.append(dice_score(pred_otsu, mask))
        otsu_jacc.append(jaccard_index(pred_otsu, mask))

        # Sauvola
        pred_sau = apply_sauvola(img, window_size=ws, k=k)
        sau_dice.append(dice_score(pred_sau, mask))
        sau_jacc.append(jaccard_index(pred_sau, mask))

    print("\n===== FINAL RESULTS =====")
    print(f"Otsu     Dice: {np.mean(otsu_dice):.4f}")
    print(f"Otsu     Jacc: {np.mean(otsu_jacc):.4f}")
    print(f"Sauvola  Dice: {np.mean(sau_dice):.4f}")
    print(f"Sauvola  Jacc: {np.mean(sau_jacc):.4f}")


## run!!!

In [ ]:
dataset = BrainMRISegmentation(
    root_dir="/data/",
    target_size=512
)


In [ ]:
# Hyperparameter search
window_sizes = [25, 51, 75, 101]
k_values = [0.15, 0.2, 0.25, 0.3]

best_params, cached_data = tune_sauvola(dataset, window_sizes, k_values)

Window=25, k=0.15, Mean Dice=0.0462


Window=25, k=0.2, Mean Dice=0.0463


Window=25, k=0.25, Mean Dice=0.0461


Window=25, k=0.3, Mean Dice=0.0458


Window=51, k=0.15, Mean Dice=0.0471


...

Window=75, k=0.3, Mean Dice=0.0510


Window=101, k=0.15, Mean Dice=0.0521


Window=101, k=0.2, Mean Dice=0.0525


Window=101, k=0.25, Mean Dice=0.0531


Window=101, k=0.3, Mean Dice=0.0537

Best Sauvola Params: (101, 0.3) Dice: 0.053672672042888836


**Best Sauvola Params: (101, 0.3) Dice: 0.053672672042888836**

In [ ]:
# Final Comparison
evaluate(dataset, best_params)

100%|██████████| 3064/3064 [02:14<00:00, 22.71it/s]


===== FINAL RESULTS =====
Otsu     Dice: 0.0707
Otsu     Jacc: 0.0376
Sauvola  Dice: 0.0537
Sauvola  Jacc: 0.0281



### FINAL RESULTS :-
```
- Otsu     Dice: 0.0707
- Sauvola  Dice: 0.0537

- Otsu     Jacc: 0.0376
- Sauvola  Jacc: 0.0281